In [ ]:
##<script>
##  jQuery(document).ready(function($) {  
##  
##  $(window).on('load', function(){
##    $('#preloader').fadeOut('slow',function(){$(this).remove();});
##  });
##  
##  });
##</script>
##
##<style type="text/css">
##  div#preloader { position: fixed; 
##      left: 0; 
##      top: 0; 
##      z-index: 999; 
##      width: 100%; 
##      height: 100%; 
##      overflow: visible; 
##      background: #fff url('http://preloaders.net/preloaders/720/Moving%20line.gif') no-repeat center center; 
##  }
##
##</style>
##
##<div id="preloader">
##
##</div>

# Food Framing Preprocessing Script 

# Contents

# Data Preparation
## [1. Load Data](#LoadData)
## [2. Eye Blink Check](#CheckBlinks)
## [3. Calculate GSF](#GSF)
## [4. Calculate DDT](#DDT)
## [5. Summary Measures](#summary)
## [6. Compute z and familiarity variables](#zfam)
## [7. Median Split](#split) 
## [8. Participant Exclusion](#PartEx)


# Data Preparation


<a id='LoadData'></a>
# 1. Loading Pilot Data
-----------------

### Load eyetracker data reports generated by EyeLink DataViewer.
#### Loading Interest Area (IA) and Saccade (Sacc) reports

In [14]:
#Select path for IA and Sacc
pathIA ="/Users/pradyumna/OneDrive - University College London/PhD Projects/FoodFramingValue/FF2018_ParticipantData/Output/IA_report_first1500ms.txt"
pathSacc ="/Users/pradyumna/OneDrive - University College London/PhD Projects/FoodFramingValue/FF2018_ParticipantData/Output/fixation_report_first1500ms.txt"
# load csv for IA and Sacc
pX_IAdata = [pd.read_csv(pathIA, sep="\t")]
pX_SACCdata = [pd.read_csv(pathSacc, sep="\t")]
# Concatenate to generate frames
IAdata = pd.concat(pX_IAdata)
SACCdata = pd.concat(pX_SACCdata)

FileNotFoundError: [Errno 2] File b'/Users/pradyumna/OneDrive - University College London/PhD Projects/FoodFramingValue/FF2018_ParticipantData/Output/IA_report_first1500ms.txt' does not exist: b'/Users/pradyumna/OneDrive - University College London/PhD Projects/FoodFramingValue/FF2018_ParticipantData/Output/IA_report_first1500ms.txt'

#### Dataframe with interest Area information

In [ ]:
IAdata

#### Dataframe with saccade information

In [13]:
SACCdata

NameError: name 'SACCdata' is not defined

## Number of participants and trials 

In [15]:
participant = IAdata['DATA_FILE'].unique()
participant_names = IAdata['DATA_FILE'].unique()

#Extract just names of participant (for folder name in future steps)
yy=0
for xx in participant_names:
    participant_names[yy]=xx.rstrip(".edf")
    yy+=1
print (participant_names)

trial_n = IAdata['trial'].unique()

print("Number of participants:", len(participant))
print ("Number of trials:", len(trial_n))

['p1' 'p5' 'p6' 'p7' 'p8' 'p9' 'p10' 'p11' 'p12' 'p13' 'p14' 'p15' 'p16'
 'p17' 'p18' 'p19' 'p20' 'p21' 'p22' 'p23' 'p24' 'p25' 'p26' 'p27' 'p28'
 'p29' 'p30' 'p31' 'p32' 'p33' 'p34' 'p35' 'p36' 'p37' 'p38' 'p39' 'p40'
 'p41' 'p42' 'p43']
Number of participants: 40
Number of trials: 240


<a id='CheckBlinks'></a>
# 2. Check Blinks across participants
------------------------------------------

#### Loading dataframe containing trial information for all the participants

In [18]:
PathTrialReport ="/Users/pradyumna/OneDrive - University College London/PhD Projects/FoodFramingValue/Reports/TrialReport_Blinks_v1.txt"
TrialRepFrame = [pd.read_csv(PathTrialReport, sep="\t")]
TrialDataFrame = pd.concat(TrialRepFrame)
#TrialDataFrame

In [19]:
# Extracting information for each participant
BlinkMatrix = []
partBlink = TrialDataFrame['DATA_FILE'].unique()
for k in range(0,len(partBlink)): 
    pX_Blinks = TrialDataFrame.loc[(TrialDataFrame['DATA_FILE']==partBlink[k]) & (TrialDataFrame['trial_type']== 'Experiment')]

    Blink_Count_trial =  pX_Blinks['BLINK_COUNT']
    Blink_Average_trial =  pd.to_numeric(pX_Blinks['AVERAGE_BLINK_DURATION'], errors='coerce')
    Duration_trial =  pd.to_numeric(pX_Blinks['DURATION'], errors='coerce')

    Blink_Count_part = Blink_Count_trial.mean()
    Blink_CountSD_part = Blink_Count_trial.std()
    Blink_Average_part = Blink_Average_trial.mean(skipna = True)
    Blink_AverageSD_part = Blink_Average_trial.std(skipna = True)
    Duration_part = Duration_trial.mean(skipna = True)

    BlinkMatrix.append([partBlink[k] , Blink_Count_part, Blink_CountSD_part, Blink_Average_part, Blink_AverageSD_part, Duration_part]) 

BlinkMatrix=pd.DataFrame(BlinkMatrix,columns=['Participant', 'BlinkCountAverage', 'BlinkCountStd', 'BlinkTimeAverage', 'BlinkTimeAverageStd','TrialDurationAverage'])

##### BlinkCountAverage : mean blink number during all the trials for each participant
##### BlinkTimeAverage : mean duration (ms) of a blink event for each participant
##### TrialDurationAverage : mean duration (ms) of the trial for each participant

In [20]:
BlinkMatrix

,Participant,BlinkCountAverage,BlinkCountStd,BlinkTimeAverage,BlinkTimeAverageStd,TrialDurationAverage
0,p1.edf,0.370833,0.672209,92.042254,80.064957,7997.291667
1,p5.edf,1.687500,1.163537,189.220090,88.097275,6377.383333
2,p6.edf,1.070833,0.881696,115.250879,38.817476,6987.525000
3,p7.edf,1.195833,1.038578,135.605978,78.495276,7628.466667
4,p8.edf,1.054167,1.128364,106.463613,17.419231,6757.916667
5,p9.edf,3.137500,1.214388,132.536250,46.917266,6479.941667
6,p10.edf,0.445833,0.624787,133.271739,71.207189,6942.641667
7,p11.edf,0.154167,0.538461,169.118929,317.848335,6862.675000
8,p12.edf,0.329167,0.610219,208.746032,123.446602,6227.416667
9,p13.edf,2.287500,1.041327,157.710462,48.511869,7254.366667


##### (*) Participant 40 (p40.edf) is the one with the highest number of blink events and the one with longer average blinking among all the participants. Additionally, he lost one trial beacuse of no fixations. p40.edf is taken out from posterior analysis.

### Preliminary subject extraction

In [22]:
#Subject 40 becuase of sleepineess during the experiment
delete_index = []
for i in range(len(participant)):
    if participant_names[i]=='p40':
        delete_index = i
print('Eliminated participant ', delete_index)
print('Eliminated participant name'  ,participant_names[delete_index])
        
participant_names = np.delete(participant_names, [delete_index], None)
participant = np.delete(participant, [delete_index], None)
participant

Eliminated participant  36
Eliminated participant name p40


array(['p1.edf', 'p5.edf', 'p6.edf', 'p7.edf', 'p8.edf', 'p9.edf',
       'p10.edf', 'p11.edf', 'p12.edf', 'p13.edf', 'p14.edf', 'p15.edf',
       'p16.edf', 'p17.edf', 'p18.edf', 'p19.edf', 'p20.edf', 'p21.edf',
       'p22.edf', 'p23.edf', 'p24.edf', 'p25.edf', 'p26.edf', 'p27.edf',
       'p28.edf', 'p29.edf', 'p30.edf', 'p31.edf', 'p32.edf', 'p33.edf',
       'p34.edf', 'p35.edf', 'p36.edf', 'p37.edf', 'p38.edf', 'p39.edf',
       'p41.edf', 'p42.edf', 'p43.edf'], dtype=object)

<a id='GSF'></a>
# 3. Calculate GSF for each participant and trial
----------------------------------

In [23]:
i=0
GSF_matrix = []

#Run for each individual participant
for i in range(len(participant)):
    #Extract the number of trials for this participant
    #pX_indSubj = SACCdata.loc[(SACCdata['DATA_FILE']==participant[i]) & (SACCdata['trial_type']== 'Experiment')]
    #Add number of trials for the experiment
    #trial_n = pX_indSubj['trial'].unique()
    trial_n = range(240)
    for  j in range(len(trial_n)):
            # extract saccade data for this participant
            GSF_data=SACCdata.loc[(SACCdata['DATA_FILE']==participant[i]) & (SACCdata['trial']== j+1) & (SACCdata['trial_type']== 'Experiment')]


            # considered direct glance transition between IA 1 (left) and IA 2 (right)
            booatstart_p1=GSF_data['CURRENT_SAC_START_INTEREST_AREA_ID'] == '2'
            booatend_p1=GSF_data['CURRENT_SAC_END_INTEREST_AREA_ID'] == '1'
            booatstart_p2=GSF_data['CURRENT_SAC_START_INTEREST_AREA_ID'] == '1'
            booatend_p2=GSF_data['CURRENT_SAC_END_INTEREST_AREA_ID'] == '2'            
            
            #Count shift only if the saccade goes from one area to the other (right to left or left to right item)
            shifts_p1= booatstart_p1 & booatend_p1
            shifts_p2= booatstart_p2 & booatend_p2

            shifts_total = shifts_p1 | shifts_p2
            #Count the total number of shifts in this trial
            trialGSF = sum(shifts_total*1)
            
            #Extract the region (left or right) for the last fixation
            go_back = 1
            tail_fix = GSF_data['CURRENT_SAC_END_INTEREST_AREA_ID'].tail(go_back)  
            
            # If we have not fixation information we don't skip it, we fill with zeros
            if GSF_data.empty:
                print ('skipped: participant:',participant[i],'trial',j+1)
                #continue
            
            ## If the sequence is empty, we assign 0 to tail
            if tail_fix.empty:
                tail_fix = 0
            else:
                tail_fix = tail_fix.values[0]
            
            while ('.'== tail_fix) | ('3'== tail_fix):
                go_back = go_back + 1
                tail_fix = GSF_data['CURRENT_SAC_END_INTEREST_AREA_ID'].tail(go_back)
                #Since tail gets the last n subjects we need to just pick the one on top
                tail_fix_aux = tail_fix.values
                tail_fix = tail_fix_aux[0]
            
            #add GSF trial info to a new row if the matrix
            GSF_matrix.append([participant[i], j+1, trialGSF, tail_fix]) #trial number, trial GSF, last_fixation
      

KeyError: 'CURRENT_SAC_START_INTEREST_AREA_ID'

In [ ]:
#Visualize GSF_matrix
#GSF_matrix

<a id='DDT'></a>
# 4. Calculate DDT for each participant and trial
-----------------------------------

## Extract measures of dwell time (DT) per Interest Area (IA) in a particular trial

In [26]:
# IA1: left item ; IA2 : right item ; IA3: condition icon
IA1_data=IAdata.loc[(IAdata['DATA_FILE']==participant[i]) & (IAdata['trial']== j+1) & (IAdata['IA_ID']== 1) & (IAdata['trial_type']== 'Experiment')]
IA2_data=IAdata.loc[(IAdata['DATA_FILE']==participant[i]) & (IAdata['trial']== j+1) & (IAdata['IA_ID']== 2) & (IAdata['trial_type']== 'Experiment')]
IA3_data=IAdata.loc[(IAdata['DATA_FILE']==participant[i]) & (IAdata['trial']== j+1) & (IAdata['IA_ID']== 3) & (IAdata['trial_type']== 'Experiment')]

print ("Information contained in IA report ")
list(IA1_data)

Information contained in IA report 


['DATA_FILE',
 'IA_ID',
 'trial',
 'Triplet_Identity',
 'trial_type',
 'LL_Item',
 'LL_Value',
 'LR_Item',
 'LR_Value',
 'Chosen_Img',
 'CHOICE_KEYPRESS',
 'CHOICE_RT',
 'CONF',
 'CONF_RT',
 'Block_Cond',
 'IA_DWELL_TIME',
 'IA_DWELL_TIME_%',
 'IA_FIXATION_COUNT',
 'IA_FIXATION_%',
 'IA_MAX_FIX_PUPIL_SIZE',
 'IA_AVERAGE_FIX_PUPIL_SIZE',
 'IA_MIN_FIX_PUPIL_SIZE',
 'IA_RUN_COUNT',
 'TRIAL_DWELL_TIME',
 'TRIAL_FIXATION_COUNT']

In [27]:
i=0
DDT_matrix = []

#Extract information for each participant. Behavioural parameters also extracted from this report.
for i in range(len(participant)):
    #pX_indSubj = IAdata.loc[(IAdata['DATA_FILE']==participant[i]) & (pX_IAdata['trial_type']== 'Experiment')]
    #Add number of trials for the experiment
    #trial_n = pX_indSubj['trial'].unique()
    trial_n = range(240)
    for  j in range(len(trial_n)):
            #Extract DT information for the trial IA1 :left ; IA2 : right
            IA1_data=IAdata.loc[(IAdata['DATA_FILE']==participant[i]) & (IAdata['trial']== j+1) & (IAdata['IA_ID']== 1) & (IAdata['trial_type']== 'Experiment')]
            IA2_data=IAdata.loc[(IAdata['DATA_FILE']==participant[i]) & (IAdata['trial']== j+1) & (IAdata['IA_ID']== 2) & (IAdata['trial_type']== 'Experiment')]

            
            #Save additional behavioural parameters
            trial_PairID = IA1_data['Triplet_Identity'].values[0]
            trial_LL_Item = IA1_data['LL_Item'].values[0]
            trial_LL_Value = IA1_data['LL_Value'].values[0]
            trial_LR_Item = IA1_data['LR_Item'].values[0]
            trial_LR_Value = IA1_data['LR_Value'].values[0]
            if IA1_data['CHOICE_KEYPRESS'].values[0] == 'm':
                trial_ChosenItm = 1  # 1: selected item is Right ; 0: selected item is Left
            else: 
                trial_ChosenItm = 0
            trial_ChoiceRT = IA1_data['CHOICE_RT'].values[0]
            trial_Conf = IA1_data['CONF'].values[0]
            trial_ConfRT = IA1_data['CONF_RT'].values[0]
            trial_BlockCond = IA1_data['Block_Cond'].values[0] #1:like 2: dislike
            
            #Calculate DDT values for the trial
            IA_1_DT = IA1_data['IA_DWELL_TIME'].values[0]
            IA_2_DT = IA2_data['IA_DWELL_TIME'].values[0]
            
            # If participant did not spend time in the Interest Areas we should skip it
            if (IA_1_DT == 0) & (IA_2_DT == 0):
                print(IA_1_DT == 0 & IA_2_DT == 0)
                print ('area1',IA_1_DT,'area2',IA_2_DT)
                print ('skipped: participant:',i,'trial',j+1)
                continue
            
            trialDDT = -IA_1_DT + IA_2_DT # since DDT_GSF_Zvalue_trial_part_v2.csv we use RIGHT - LEFT
            
            #define matrix DDT + behavioural parameters
            DDT_matrix.append([i+1, j+1, trial_PairID, trial_LL_Item, trial_LL_Value, trial_LR_Item, trial_LR_Value, trial_ChosenItm, trial_ChoiceRT, trial_Conf, trial_ConfRT, trial_BlockCond, trialDDT,IA_1_DT, IA_2_DT]) 

#list(DDT_matrix)

True
area1 0 area2 0
skipped: participant: 1 trial 167
True
area1 0 area2 0
skipped: participant: 1 trial 168
True
area1 0 area2 0
skipped: participant: 1 trial 169
True
area1 0 area2 0
skipped: participant: 1 trial 208
True
area1 0 area2 0
skipped: participant: 1 trial 212
True
area1 0 area2 0
skipped: participant: 12 trial 10
True
area1 0 area2 0
skipped: participant: 12 trial 32
True
area1 0 area2 0
skipped: participant: 14 trial 121
True
area1 0 area2 0
skipped: participant: 14 trial 220
True
area1 0 area2 0
skipped: participant: 18 trial 120
True
area1 0 area2 0
skipped: participant: 18 trial 228
True
area1 0 area2 0
skipped: participant: 26 trial 51
True
area1 0 area2 0
skipped: participant: 29 trial 135
True
area1 0 area2 0
skipped: participant: 29 trial 201
True
area1 0 area2 0
skipped: participant: 29 trial 206
True
area1 0 area2 0
skipped: participant: 38 trial 10
True
area1 0 area2 0
skipped: participant: 38 trial 94


#### Stack both results (GSF and DDT) matrices together

In [34]:
DDT_GSF_trial = np.column_stack((DDT_matrix))

#### Insert headers

In [ ]:
DDT_GSF_head = np.insert(DDT_matrix,  0, np.array(('Part', 'TrialN', 'PairID', 'LItem', 'LValue', 'RItem', 'RValue', 'ChosenITM', 'ChoiceRT', 'Conf', 'ConfRT', 'BlockCond', 'tDDT', 'lIA_DT', 'rIA_DT')), axis=0)

<a id='summary'></a>
# 5. Display Summary Measures
----------------------------------------

In [ ]:
#Generate DataFrame for new summary table                                              
DataFrameSum1=pd.DataFrame(DDT_GSF_head,columns=['Part', 'TrialN', 'PairID', 'LItem', 'LValue', 'RItem', 'RValue', 'ChosenITM', 'ChoiceRT', 'Conf', 'ConfRT', 'BlockCond', 'tDDT', 'lIA_DT', 'rIA_DT', 'tGSF','lastFixat'])
DataFrameSum1=DataFrameSum1.drop(DataFrameSum1.index[0])

## Confidence distributions

In [ ]:
participant_densities(DataFrameSum1,'Conf')

## Choice RT distributions

In [ ]:
participant_densities(DataFrameSum1,'ChoiceRT',)

## Trial DDT distributions

In [ ]:
participant_densities(DataFrameSum1,'tDDT')

##  Item value distributions

### Right and left distributions are identical since we have the permutation

In [ ]:
participant_densities(DataFrameSum1,'RValue') # Distribution of value of items 

## Trial GSF

In [ ]:
participant_densities(DataFrameSum1,'tGSF')

<a id='zfam'></a>
# 6. Calculate Z-scores 
---------------------------------------------

In [ ]:
# Calculate Z-values and other 
z_matrix=[]
z_matrix_aux=[]

for i in (DataFrameSum1['Part'].unique()):
    Choicedata = DataFrameSum1.loc[DataFrameSum1['Part'] == str(i)]
    # from v3 and on we define DV as  RIGHT - LEFT values of the items
    pX_DV = - pd.to_numeric(Choicedata["LValue"]) + pd.to_numeric(Choicedata["RValue"])
    pX_zDV = (pX_DV - np.mean(pX_DV))/np.std(pX_DV)
    pX_absDV = abs(- pd.to_numeric(Choicedata['LValue']) + pd.to_numeric(Choicedata['RValue']))
    pX_zAbsDV = (pX_absDV - np.mean(pX_absDV))/np.std(pX_absDV)
    pX_zConf = (pd.to_numeric(Choicedata['Conf']) - np.mean(pd.to_numeric(Choicedata['Conf'])))/np.std(pd.to_numeric(Choicedata['Conf']))
    pX_zChoiceRT = (pd.to_numeric(Choicedata['ChoiceRT'] )- np.mean(pd.to_numeric(Choicedata['ChoiceRT'])))/np.std(pd.to_numeric(Choicedata['ChoiceRT']))
        
    z_matrix_aux= np.column_stack((pX_DV,pX_zDV,pX_absDV,pX_zAbsDV,pX_zConf,pX_zChoiceRT))
    
    for  j in range(len(z_matrix_aux)):    
        z_matrix.append(z_matrix_aux[j])

In [ ]:
# Add to aggregate table
Final_trial = np.column_stack((DDT_GSF_trial,z_matrix))
Final_head = np.insert(Final_trial,  0, np.array(('Part', 'TrialN', 'PairID', 'LItem', 'LValue', 'RItem', 'RValue', 'ChosenITM', 'ChoiceRT', 'Conf', 'ConfRT', 'BlockCond', 'tDDT', 'lIA_DT', 'rIA_DT', 'tGSF','lastFixat','DV','zDV','absDV','zAbsDV','zConf','zChoiceRT')), axis=0)

In [ ]:
#save text file with DDT and GSF for each participant/trial
#np.savetxt("DataFoodFramingNotebook_v1.csv", Final_head, delimiter=",", fmt='%s')

## Extract familiarity information from the BDM and add it to each trial in the choice step of the experiment

## Load choice data

In [ ]:
#Load choice trial values (saved in previous stage)

#LOAD FROM FILE
#data_exp1 = pd.read_csv('/Users/bdmlab/Google Drive/BdM Projects/FoodFraming_Amy/Analysis Scripts/DataFoodFramingNotebook_v1.csv') # The main data file for the first

#CONTINUE USING DATASET FROM PREVIOUS STAGE
data_exp1 = pd.DataFrame(Final_trial,columns = ['Part', 'TrialN', 'PairID', 'LItem', 'LValue', 'RItem', 'RValue', 'ChosenITM', 'ChoiceRT', 'Conf', 'ConfRT', 'BlockCond', 'tDDT', 'lIA_DT', 'rIA_DT', 'tGSF','lastFixat','DV','zDV','absDV','zAbsDV','zConf','zChoiceRT'])
data_exp1.head()

## Load data in python - BDM data

In [ ]:
#Extract BDM data from all the subject folders (Separated folder for each participant, each one containing the file "BDM_RESULTS_FILE.txt")

#INPUT PATH FOR BDM_RESULTS_FILE folder
main_path = "/Users/bdmlab/OneDrive - University College London/PhD Projects/FoodFramingValue/Food Framing Data Raw/"
file_list = os.listdir(main_path)

print("File List in Folder")
print(sorted(file_list))

In [ ]:
file_array = []

for k in range(len(participant_names)):
        BDM_filename = "/BDM_RESULTS_FILE.txt"
        files =  main_path + participant_names[k] + BDM_filename
        file_array.append(files) 
        
print("number of files to load:" ,len(file_array))        
#file_array
#Make sure that the paths are properly defined 

In [ ]:
#define partipant names(numbers)
participant = (data_exp1['Part'].unique())
participant

### Adding familiarity values and new variables

In [ ]:
FamLeft  = []
FamRight = []
Tot_Fam = []
DFam = []
FamChosen = []
FamUnChosen = []
ValChosen = []
ValUnChosen = []
Tot_Val = []
Correct_val = []
correct = []

LValConfBDM = []
LFamConfBDM = []
RValConfBDM = []
RFamConfBDM = []

trial_count = 0

#for cycle for participant
for k in range(0,len(file_array)):
    BDMframe = [pd.read_csv(file_array[k], sep="\t")]
    BDMdata = pd.concat(BDMframe)
    
    #for cycle for trials 
    data_part = data_exp1.loc[(data_exp1['Part'] == str(participant[k]))]
    trial_n = data_part['TrialN'].unique()
    #print(data_part)

    for  j in range(len(trial_n)):    
        
        #Pick identity left item  
        L_item_name = data_part['LItem'][trial_count]
        FamValue_L = BDMdata.loc[(BDMdata['BDM_ITM']== L_item_name),'BDM_FAM'].values[0]
        Value_L = BDMdata.loc[(BDMdata['BDM_ITM']== L_item_name),'BDM_VAL'].values[0]
        L_ValConfBDM = BDMdata.loc[(BDMdata['BDM_ITM']== L_item_name),'BDM_CONF'].values[0]
        L_FamConfBDM = BDMdata.loc[(BDMdata['BDM_ITM']== L_item_name),'BDM_FAMCONF'].values[0]

        #Pick identity right item
        R_item_name = data_part['RItem'][trial_count]
        FamValue_R = BDMdata.loc[(BDMdata['BDM_ITM']== R_item_name),'BDM_FAM'].values[0]
        Value_R = BDMdata.loc[(BDMdata['BDM_ITM']== R_item_name),'BDM_VAL'].values[0]
        R_ValConfBDM = BDMdata.loc[(BDMdata['BDM_ITM']== R_item_name),'BDM_CONF'].values[0]
        R_FamConfBDM = BDMdata.loc[(BDMdata['BDM_ITM']== R_item_name),'BDM_FAMCONF'].values[0]

        
        BDMFamValue = BDMdata.loc[(BDMdata['BDM_ITM']== R_item_name),'BDM_FAM'].values[0]

        ChosenITMtrial = data_part['ChosenITM'][trial_count]
        BlockCondtrial = data_part['BlockCond'][trial_count]

        if ChosenITMtrial == "1":  #Selected Item is Right
            FamChosenT  = FamValue_R
            FamUnChosenT  = FamValue_L
            ValUnChosenT  = Value_L
            ValChosenT  = Value_R
            
            if BlockCondtrial == "1.0": # Block is like, correct chosen is higher value
                if Value_R >= Value_L:
                    correct = 1
                else: 
                    correct = 0
            
            if BlockCondtrial == "2.0" :# Block is dislike, correct chosen is lower value
                if Value_L >= Value_R:
                    correct = 1
                else: 
                    correct = 0  
                
        else: # Selected item is Left
            FamChosenT  = FamValue_L
            FamUnChosenT  = FamValue_R
            ValUnChosenT  = Value_R           
            ValChosenT = Value_L
            
            if BlockCondtrial == "1.0": # Block is like, correct chosen is higher value
                if Value_L >= Value_R:
                    correct = 1
                else: 
                    correct = 0
            
            if BlockCondtrial == "2.0" :# Block is dislike, correct chosen is lower value
                if Value_R >= Value_L:
                    correct = 1
                else: 
                    correct = 0  
        
        FamValTot = FamValue_L + FamValue_R
        DFamT =  FamValue_R - FamValue_L 
        
        
        ValTot = Value_L + Value_R
        
        #add to the rows with current trial measures
        FamLeft.append(str(FamValue_L))
        FamRight.append(str(FamValue_R))
        Tot_Fam.append(str(FamValTot))
        DFam.append(str(DFamT))
        Tot_Val.append(str(ValTot))    
        FamChosen.append(str(FamChosenT))
        FamUnChosen.append(str(FamUnChosenT))
        ValChosen.append(str(ValChosenT))
        ValUnChosen.append(str(ValUnChosenT))
        Correct_val.append(str(correct))
        
        LValConfBDM.append(str(L_ValConfBDM))
        LFamConfBDM.append(str(L_FamConfBDM))
        RValConfBDM.append(str(R_ValConfBDM))
        RFamConfBDM.append(str(R_FamConfBDM))
        
        
        
        trial_count = trial_count + 1
        


## Extract familiarity information from the BDM and add it to each trial in the choice step of the experiment

#### Stack columns for the new familiarity values

In [ ]:
table_aux= np.column_stack((DFam,FamLeft,FamRight,Tot_Fam,Tot_Val,FamChosen,FamUnChosen,ValChosen,ValUnChosen,Correct_val,LValConfBDM,LFamConfBDM,RValConfBDM,RFamConfBDM))

In [ ]:
# Check similar size in both tables
print(len(table_aux))
print(len(data_exp1))

In [ ]:
Fam_part = pd.DataFrame(table_aux,columns=['DFam','LFam','RFam','TotFam','TotVal','FamCh','FamUnCh','ValCh','ValUnCh','Correct','LValConfBDM','LFamConfBDM','RValConfBDM','RFamConfBDM'])
new_data = pd.concat([data_exp1, Fam_part],axis =1)

#### Calculate Z-values for familiarity and (un)Chosen values

In [ ]:
# Agregate data matrix
new_data = new_data.apply(pd.to_numeric, errors='ignore')
z_matrix=[]
z_matrix_aux=[]

for i in unique(new_data["Part"]):
    Choicedata = new_data.loc[new_data['Part'] == i]
    
    # Z-score right and left item values 
    pX_LValue = pd.to_numeric(Choicedata['LValue'])
    zLValue = (pX_LValue - np.mean(pX_LValue))/np.std(pX_LValue)
    
    pX_RValue = pd.to_numeric(Choicedata['RValue'])
    zRValue = (pX_RValue - np.mean(pX_RValue))/np.std(pX_RValue)
    
    # Z-score for BDM confidence (value and familiarity)
    pX_LValConfBDM = pd.to_numeric(Choicedata['LValConfBDM'])
    zLValConfBDM = (pX_LValConfBDM - np.mean(pX_LValConfBDM))/np.std(pX_LValConfBDM)
    
    pX_LFamConfBDM = pd.to_numeric(Choicedata['LFamConfBDM'])
    zLFamConfBDM = (pX_LFamConfBDM - np.mean(pX_LFamConfBDM))/np.std(pX_LFamConfBDM)
    
    pX_RValConfBDM = pd.to_numeric(Choicedata['RValConfBDM'])
    zRValConfBDM = (pX_RValConfBDM - np.mean(pX_RValConfBDM))/np.std(pX_RValConfBDM)
    
    pX_RFamConfBDM = pd.to_numeric(Choicedata['RFamConfBDM'])
    zRFamConfBDM = (pX_RFamConfBDM - np.mean(pX_RFamConfBDM))/np.std(pX_RFamConfBDM)
    
    # Z-score familiarity 
    pX_LFam = pd.to_numeric(Choicedata['LFam'])
    zLFam = (pX_LFam - np.mean(pX_LFam))/np.std(pX_LFam)
    
    pX_RFam = pd.to_numeric(Choicedata['RFam'])
    zRFam = (pX_RFam - np.mean(pX_RFam))/np.std(pX_RFam)
    
    pX_DFam = pd.to_numeric(Choicedata['DFam'])
    zDFam = (pX_DFam - np.mean(pX_DFam))/np.std(pX_DFam)
    
    pX_totFam = pd.to_numeric(Choicedata['TotFam'])
    z_totFam = (pX_totFam - np.mean(pX_totFam))/np.std(pX_totFam)

    pX_totVal = pd.to_numeric(Choicedata['TotVal'])
    z_totVal = (pX_totVal - np.mean(pX_totVal))/np.std(pX_totVal)
    
    pX_ValChosen = pd.to_numeric(Choicedata['ValCh'])
    z_ValChosen = (pX_ValChosen - np.mean(pX_ValChosen))/np.std(pX_ValChosen)

    pX_ValUnChosen = pd.to_numeric(Choicedata['ValUnCh'])
    z_ValUnChosen = (pX_ValUnChosen - np.mean(pX_ValUnChosen))/np.std(pX_ValUnChosen)

    pX_FamChosen = pd.to_numeric(Choicedata['FamCh'])
    z_FamChosen = (pX_FamChosen - np.mean(pX_FamChosen))/np.std(pX_FamChosen)

    pX_FamUnChosen = pd.to_numeric(Choicedata['FamUnCh'])
    z_FamUnChosen = (pX_FamUnChosen - np.mean(pX_FamUnChosen))/np.std(pX_FamUnChosen)
    
    pX_tGSF = pd.to_numeric(Choicedata['tGSF'])
    z_tGSF = (pX_tGSF - np.mean(pX_tGSF))/np.std(pX_tGSF)
    
    pX_tDDT = pd.to_numeric(Choicedata['tDDT'])
    z_tDDT = (pX_tDDT - np.mean(pX_tDDT))/np.std(pX_tDDT)

    absDDT = abs(z_tDDT)
    abszDFam = abs(zDFam)


    
    z_matrix_aux= np.column_stack((zDFam,z_totFam,z_totVal,z_ValChosen,z_ValUnChosen,z_FamChosen,z_FamUnChosen, z_tGSF, z_tDDT, absDDT,abszDFam,zLValue,zRValue,zLFam,zRFam,zLValConfBDM,zLFamConfBDM,zRValConfBDM,zRFamConfBDM))
    
    for  j in range(len(z_matrix_aux)):    
        z_matrix.append(z_matrix_aux[j])

In [ ]:
Zmatrix_part = pd.DataFrame(z_matrix,columns=['zDFam','zTotFam','zTotVal','zValCh','zValUnCh','zFamCh','zFamUnCh', 'zGSF', 'zDDT', 'absDDT','zAbsDFam','zLValue','zRValue','zLFam','zRFam','zLValConfBDM','zLFamConfBDM','zRValConfBDM','zRFamConfBDM'])

<a id='split'></a>
#  7. Including Median Split Data
----------------------------------

#### Calculate median split for each participant and block conditions separatedly. 
#### We apply a split separating participants and block conditions

### Insert the name of the variables to be used for the median split

In [ ]:
Splits_headers = ['ConfSplit', 'GSFMedSplit','DDTMedSplit']
Splits_variables = ['Conf', 'tGSF','tDDT']

In [ ]:
median_matrix=[]
median_matrix_aux=[]
median_matrix_aux_cond=[]
Split_cols = []

for h in range(len(Splits_variables)):
    median_matrix=[]
    for i in unique(new_data["Part"]):
        median_matrix_aux=[]

        for j in unique(new_data["BlockCond"]):
            #median_matrix_aux_cond=[]

            #Extract data by participant and BlockCond (1: Like, 2: Dislike)
            Splitdata = new_data.loc[new_data['Part'] == i]
            Splitdata = Splitdata.loc[Splitdata['BlockCond'] == j]
            
            # Select the column to split
            Splitdata = Splitdata.loc[:,['TrialN',Splits_variables[h]]]
            SplitSort = Splitdata.values
            # Sort for confidence column
            SplitSort = SplitSort[np.argsort(SplitSort[:, 1])]
            #print(SplitSort)
            lenPart = len(Splitdata)
            #print(lenPart)
            # Generate column with median order
            median_low = [0] * int (lenPart/2)
            #print (median_low)
            median_high = [1] * int (lenPart/2)
            #print (median_high)
            median_split_order = median_low + median_high
            #Attach split median order column (1 and 0)
            median_aux= np.column_stack((SplitSort,median_split_order))
            #Sort back to trial number
            SplitSorted = median_aux[np.argsort(median_aux[:, 0])]
            #print(len(SplitSorted))
            
            for kk in range(len(SplitSorted)): 
                if len(median_matrix_aux) == 0 :
                    median_matrix_aux = SplitSorted[0]
                else:
                    median_matrix_aux = np.vstack([median_matrix_aux, SplitSorted[kk]])
        
        #print(len(median_matrix_aux))
        # sort after adding both conditions    
        median_matrix_aux = median_matrix_aux[np.argsort(median_matrix_aux[:, 0])]
        
        #Extract only the column with the median split information
        median_matrix_aux = median_matrix_aux[:,2]
        
        #append to median matrix for the participants
        for k in range(len(median_matrix_aux)):    
            median_matrix.append(median_matrix_aux [k])
    
    median_matrix = pd.DataFrame(median_matrix,columns=[Splits_headers[h]])
    #print(len (median_matrix))
    #add new column (new split variable columns) to the split matrix
    if  h == 0:
        Split_cols = median_matrix
    else:
        Split_cols = pd.concat([Split_cols,median_matrix ],axis =1)


        

In [ ]:
#Join new familiarity and split datasets with the old trial information
data_exp1 = pd.concat([new_data, Zmatrix_part, Split_cols],axis =1)


In [ ]:
# Consolidate variable names in Dataframe structure
data_exp1.rename(index=str, columns={"A": "a", "B": "c"})


In [ ]:
#save text file with DDT and GSF for each participant/trial
data_exp1.to_csv("Output/DataFoodFramingNotebook_BDMConfs_v2.csv")


<a id='PartEx'></a>
# 8. Participant Exclusion
---------------------------------------------------------------------------------------

##  Exclusion Criteria
### As used in Folke et al. (2016)

#### (1) Participants used less than 25% of the BDM scale.
#### (2) Participants gave exactly the same BDM rating for more than 25% of the items.  [For our study adapted to 50%]
#### (3) Participants used less than 25% of the confidence scale.
#### (4) Participants gave exactly the same confidence rating for more than 25% of their choices. [For our study adapted to 50%]
#### (*) adpted values to avoid eliminating too many participants from our study. Additionally, we consider that our slider moves in bigger jumps than the one used by Tomas.


In [ ]:
ExclusionMatrix = []
#for cycle for participant
for k in range(0,len(file_array)):
 
    # Checking for values 
    BDMframe = [pd.read_csv(file_array[k], sep="\t")]
    BDMframe = pd.concat(BDMframe)
    BDMvalue = BDMframe['BDM_VAL']
    ValueRange = max(BDMvalue) - min(BDMvalue)
    ValueCounts = BDMvalue.value_counts()
    maxValueCounts = max(ValueCounts)
    ValueCountsTotal = sum(int(i) for i in ValueCounts.values)
    
    PropnMostFreqVal = maxValueCounts / ValueCountsTotal
    PropnRangeVal = ValueRange / 3
    
    # Checking for confidence
    participant_data = data_exp1.loc[ (data_exp1['Part'] == k+1)]
    ConfidencePart =   participant_data ['Conf']
    
    ConfRange = max(ConfidencePart) - min(ConfidencePart)
    ConfCounts = ConfidencePart.value_counts()
    
    maxConfCounts = max(ConfCounts)
    ConfCountsTotal = sum(int(i) for i in ConfCounts.values)
    
    PropnMostFreqConf = maxConfCounts / ConfCountsTotal
    PropnRangeConf = ConfRange / 100

    
    ExclusionMatrix.append([k+1 , PropnMostFreqVal, PropnRangeVal, PropnMostFreqConf, PropnRangeConf]) 

#ExclusionMatrix=pd.DataFrame(ExclusionMatrix,columns=['Participant', 'ProportionMostFrequentBid', 'ProportionRangeBid','ProportionMostFrequentConfidence', 'ProportionRangeConfidence])

##### ProportionMostFrequentBid : the proportion of all the BDM bids that were reported with the most frequent price (0: no repetition, 1: all items received the same bid price )
##### ProportionRangeBid : proportion of the whole range (£0 - £3) that was covered by the reported BDM bid price (0: no range, 1: whole range covered)
##### ProportionMostFrequentConfidence : the proportion of all the confidence values (during the CHOICE STAGE) that has been reported with a single (most frequent) value (0: no repetition, 1: all items received the same confidence value )
##### ProportionRangeConfidence : proportion of the whole range (0 - 100) that was covered by the confidence values (0: no range, 1: whole range covered)

In [ ]:
ExclusionMatrix=pd.DataFrame(ExclusionMatrix,columns=['Participant', 'ProportionMostFrequentBid', 'ProportionRangeBid','ProportionMostFrequentConfidence', 'ProportionRangeConfidence'])
ExclusionMatrix

### Excluded Participants

#### -Participant 6 excluded by criteria (1)
#### -Participant 33 excluded by criteria (2)
#### -NO participants excluded by criteria (3)
#### -Participants 7, 17, 25, 30  excluded by criteria (4)


In [ ]:
# Shortcut, load previous data (to avoid running from the top)
# data_exp1 = pd.read_csv('/Users/pradyumna/Documents/gitDocs/FF2018/Output/DataFoodFramingNotebook_v1.csv') 
# data_exp1 = pd.read_csv('/Users/bdmlab/Documents/GiTs/FF2018/Output/DataFoodFramingNotebook_v1.csv') 

for i in [6, 7, 17, 25, 30, 33]:
    data_exp1 = data_exp1[(data_exp1.Part != i)]
data_exp1["Part"].unique()

In [ ]:
len(data_exp1["Part"].unique())

In [ ]:
data_exp1.to_csv("Output/DataFoodFramingNotebook_ExcludedPart_v1.csv")

#### *Additionally, it was checked that trials in which participants did not fixate in any of the items were were skipped (trials fulfiling this criteria were found only in participant 'p40.edf')*

# Appendix
_____________________________________________________________________________________________________________________

<a id='ImpPkg'></a>
# 1. Import Packages

In [1]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame as DF
import seaborn as sns
sns.set(color_codes=True)
#np.random.seed(sum(map(ord, "distributions")))
from sklearn import linear_model  # packages for the logistic regression function to plot the logistic regression 
from sklearn.linear_model import LogisticRegression # packages for the logistic regression function to plot the logistic regression 
import scipy
from scipy import stats, integrate
from scipy.stats import mode
from scipy.stats.stats import pearsonr # Pearson's correlation
from copy import copy as copy
import operator as operator
import pylab

# Plotting tools
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%pylab inline
figsize(5, 5)

import glob
import os

Populating the interactive namespace from numpy and matplotlib


/Users/pradyumna/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy', 'pylab']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
import rpy2


%load_ext rpy2.ipython
# Set up interface with R
# Make it easy to set and find values in a multi-index DF
idx = pd.IndexSlice

# Set up interface with R
%config InlineBackend.figure_format = 'retina'
import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore", category=RRuntimeWarning)


In [3]:
%%R
# Use in case the libraries are not available in the system
#install.packages("lme4")
#install.packages("car")
#install.packages("ggplot2")
#install.packages("broom")
#install.packages("arm")
#install.packages("ggplot2")
#install.packages("optimx")
#install.packages("multcomp")

NULL


In [4]:
%%R
library(lme4)
library(car)
library(optimx)
library(ggplot2)
library(MASS)
library(pbkrtest)
library(broom)
library(dplyr)
library(dplyr)
library(reshape2)
library(arm)
library(multcomp)


Error in library(ggplot2) : there is no package called ‘ggplot2’


<a id='LoadFunc'></a>
# 2. Defining Functions

#### Taken from Folke et al. (2016)

### Plot distributions

In [5]:
def participant_densities(data, var, xlim=(0,100), rug=True):
    # a counter that tells us where a given participant's data should be plotted
    order = 1

    # a list of all the participants in the dataset
    participants = data.loc[:, 'Part'].unique()

    # defining the figure size
    sns.set_style('white')
    fig = figure(figsize=(15,70))

    for x in participants:
        # defining the sub figures
            sub={}
            sub['%s' % x] = plt.subplot(len(participants)/2, 3, order)
            sns.kdeplot(data.loc[data['Part'] == x, var].values, ax = sub['%s' % x], shade=True)
            #if rug==True:
            #    sns.rugplot(data.loc[data['Part'] == x, var].values, ax = sub['%s' % x])
            sub['%s' % x].set_title('participant %s' % x)
            #sub['%s' % x].set_xlim(xlim)
            order += 1

### Split variable into participantwise quantiles

In [6]:
def parsplit(DF, input, quantiles):
    qvalues = pd.qcut(DF[input], quantiles, labels = range(1, quantiles+1))
    return qvalues

### Full simple logistic graph

In [7]:
def logisticplot_all (moderator, modhigh, modlow, data, xaxis='zDV', yaxis='G_choice', ylab='P(Chose Reference Item)', xlab='DV (Z-score)',
                 modhighcol='#000000', modlowcol='#AAAAAA', title='empty', parvar='SubNo'):
    
    sns.set(font_scale=1.5, style='white')
    fig = figure(figsize=(8,7))
    fig.set_facecolor('white')
    
    # defining the sigmoid function
    def model(x):
        y = 1 / (1 + np.exp(-x))
        return y
    
    sub = plt.subplot()


    #run the classifier
    clf = linear_model.LogisticRegression(C=1e5)

    # Paula used these dictionaries to store the values of the predictive lines for all the participants.
    logit_low = {}
    logit_high = {}

    # I think this defines the problem space
    X_test = np.linspace(-5,10,300)

    # fitting the predictive logistic model for the low_confidence trials, for a participant specified by x
    # first I specify the value difference right - left, then I specify the choices, left or right
    clf.fit(data.loc[data[(data[moderator]==0)].index, xaxis][:, np.newaxis],
            data.loc[data[(data[moderator]==0)].index, yaxis])
    logit_low = model(X_test*clf.coef_ + clf.intercept_).ravel()


    # fitting the predictive logistic model for the high_confidence trials, for a participant specified by x
    # first I specify the value difference right - left, then I specify the choices, left or right
    clf.fit(data.loc[data[(data[moderator]==1)].index, xaxis][:, np.newaxis],
            data.loc[data[(data[moderator]==1)].index, yaxis])
    logit_high = model(X_test * clf.coef_ + clf.intercept_).ravel()

    #Plotting the predictive lines
    line_high = sub.plot(X_test, logit_high, color=modhighcol, linewidth=5, label=modhigh, zorder=6)
    line_low = sub.plot(X_test, logit_low, color=modlowcol, linewidth=5, label=modlow, zorder=5)


    #Plotting the binned data
    data['DVBin2'] = data.groupby(parvar).apply(parsplit, input=xaxis, quantiles=4).values
    
    # determine the x coordinates
    x_cords= data.groupby('DVBin2')[xaxis].mean()
    
    # determine low y coordinates
    y_cords_low = data.loc[(data[moderator]==0), :].groupby('DVBin2')[yaxis].mean().values
    
    # determine low y standard errors
    test = pd.DataFrame(data.loc[(data[moderator]==0), :].groupby(['DVBin2', parvar])[yaxis].mean()).reset_index()
    y_low_error = test.groupby('DVBin2')[yaxis].std()/np.sqrt(len(test[parvar].unique()))
    
    
    # determine high y coordinates
    y_cords_high = data.loc[(data[moderator]==1), :].groupby('DVBin2')[yaxis].mean().values
    
    # determine high y standard errors
    test2 = pd.DataFrame(data.loc[data[moderator]==1, :].groupby(['DVBin2', parvar])[yaxis].mean()).reset_index()
    y_high_error = test2.groupby('DVBin2')[yaxis].std()/np.sqrt(len(test[parvar].unique()))
    
    # plot the low points
    plt.scatter(x_cords, y_cords_low, c=modlowcol, marker='D', s=60, zorder=1)
    # plot low error bars
    plt.errorbar(x_cords, y_cords_low, yerr=y_low_error, fmt='o', zorder=3, c=modlowcol)
    
    # plot the high points
    plt.scatter(x_cords, y_cords_high, c=modhighcol, marker='o', s=60, zorder=2)
    # plot high error bars
    plt.errorbar(x_cords, y_cords_high, yerr=y_high_error, fmt='o', zorder=4, c=modhighcol)
    
    
    # Set Labels
    sub.set_ylabel(ylab, fontsize=30)
    sub.set_xlabel(xlab, fontsize=30)

    # Set Ticks
    sub.set_xticks((-5,-3,-1,1,3,5))
    sub.set_yticks((0,0.25,0.5,0.75,1))
    sub.tick_params(axis='both', which='major', labelsize=20)

    # Set Limits
    sub.set_ylim(-0.01, 1.01)
    sub.set_xlim(-5, 5)

    # Set Title
    if title == 'empty':
        sub.set_title('')
    else:
        sub.set_title(title)
    
    sub.legend(loc=2, prop={'size':20})
    sns.despine()

### Full simple logistic graph (no bins)

In [8]:
def logisticplot_simpl (moderator, modhigh, modlow, data, xaxis='zDV', yaxis='G_choice', ylab='P(Chose Reference Item)', xlab='DV (Z-score)',
                 modhighcol='#000000', modlowcol='#AAAAAA', title='empty', parvar='SubNo'):
    
    sns.set(font_scale=1.5, style='white')
    fig = figure(figsize=(8,7))
    fig.set_facecolor('white')
    
    # defining the sigmoid function
    def model(x):
        y = 1 / (1 + np.exp(-x))
        return y
    
    sub = plt.subplot()


    #run the classifier
    clf = linear_model.LogisticRegression(C=1e5)

    # Paula used these dictionaries to store the values of the predictive lines for all the participants.
    logit_low = {}
    logit_high = {}

    # I think this defines the problem space
    X_test = np.linspace(-5,10,300)

    # fitting the predictive logistic model for the low_confidence trials, for a participant specified by x
    # first I specify the value difference right - left, then I specify the choices, left or right
    clf.fit(data.loc[data[(data[moderator]==0)].index, xaxis][:, np.newaxis],
            data.loc[data[(data[moderator]==0)].index, yaxis])
    logit_low = model(X_test*clf.coef_ + clf.intercept_).ravel()
    print ('Low measure coef',clf.coef_)
    
    # fitting the predictive logistic model for the high_confidence trials, for a participant specified by x
    # first I specify the value difference right - left, then I specify the choices, left or right
    clf.fit(data.loc[data[(data[moderator]==1)].index, xaxis][:, np.newaxis],
            data.loc[data[(data[moderator]==1)].index, yaxis])
    logit_high = model(X_test * clf.coef_ + clf.intercept_).ravel()
    print ('High measure coef',clf.coef_)



    #Plotting the predictive lines
    line_high = sub.plot(X_test, logit_high, color=modhighcol, linewidth=5, label=modhigh, zorder=6)
    line_low = sub.plot(X_test, logit_low, color=modlowcol, linewidth=5, label=modlow, zorder=5) 
    
    # Set Labels
    sub.set_ylabel(ylab, fontsize=30)
    sub.set_xlabel(xlab, fontsize=30)

    # Set Ticks
    sub.set_xticks((-5,-3,-1,1,3,5))
    sub.set_yticks((0,0.25,0.5,0.75,1))
    sub.tick_params(axis='both', which='major', labelsize=20)

    # Set Limits
    sub.set_ylim(-0.01, 1.01)
    sub.set_xlim(-5, 5)

    # Set Title
    if title == 'empty':
        sub.set_title('')
    else:
        sub.set_title(title)
    
    sub.legend(loc=2, prop={'size':20})
    sns.despine()

### Regression Coefficients Plot

In [9]:
def Coefpresplot(regtable, intercept=False, title='Regression Coefficients', size='big', ylimits=(), ymultiple=0.5, ticklabsize=25, n_ET_predictors = 0):

    # Import itertools so that we can iterate through the colours
    import itertools
    
    # Import locators so that we can tidy up the yaxis
    from matplotlib.ticker import MultipleLocator, FormatStrFormatter
    
    # rounding function to get edges to even 0.5 values
    def round_to(n, precision):
        correction = 0.5 if n >= 0 else -0.5
        return int( n/precision+correction ) * precision

    def round_to_5(n):
        return round_to(n, 0.5)

    # Set seaborn style for the plot
    sns.set(style='white')
    
    # Generate the figure
    if size=='big':
        fig = plt.figure(figsize=[20,8])
    elif size=='long':
        fig = plt.figure(figsize=[20,4])
    elif size=='narrow':
        fig = plt.figure(figsize=[10,4])
        
    fig.suptitle(title, fontsize=20)
    gs = GridSpec(1,1,bottom=0.18,left=0.18,right=0.82)
    ax = fig.add_subplot(gs[0,0])
    
    # Set axis limits based on whether to include the intercept or not    
    if intercept == True:
        XLim = (0.75, len(regtable.columns) + 0.25)
        YLim = (round_to_5(regtable.loc['CImin', :].min()-0.1), round_to_5(regtable.loc['CImax', :].max()+0.2))
    else:
        XLim = (0.75, len(regtable.columns) - 0.75)
        YLim = (round_to_5(regtable.loc['CImin', regtable.columns[1]:].min()-0.2), round_to_5(regtable.loc['CImax', regtable.columns[1]:].max()+0.2))
    if ylimits != ():
        YLim = ylimits
    ax.set_xlim(XLim)
    ax.set_ylim(YLim)
    
    # Draw a line through the 0-value on the y-axis
    line = ax.plot(XLim, [0, 0], color='black', ls='--', alpha = 0.5, lw=3)
    
    
    # If intercept is true, plot the coefficient for the intercept
    if intercept == True:
        Coefficients = regtable.columns
    else:
        Coefficients = regtable.columns[1:]
        
    # Determine the colours for the coefficients based on the n_ET_variable
    n_predictors = len(Coefficients)
    n_non_ET_predictors = n_predictors - n_ET_predictors
    
    colourlist = ['#000000'] * n_non_ET_predictors + ['#03719c'] * n_ET_predictors
    
        
    # Plot all the coefficients with 95% CI
    position = 0
    for Coefficient in Coefficients:
        position += 1
        ax.plot(position, regtable.loc['coefficient', Coefficient], marker='o', ms=8, color=colourlist[position-1],)
        ax.errorbar(position, regtable.loc['coefficient', Coefficient],
                    yerr=regtable.loc['se', Coefficient]*1.96, lw=2, color=colourlist[position-1])

    # Setting the x-axis major tick's location
    ax.set_xticks(range(1, position+1))
    
    # set the y-axis major tick position
    ax.yaxis.set_major_locator(MultipleLocator(ymultiple))


    # Setting the x-axis major tick's label
    if intercept == True:
        ax.set_xticklabels(regtable.columns, rotation=40)
    else: 
        ax.set_xticklabels(regtable.columns[1:], rotation=40)
    
    ax.tick_params(axis='both', which='major', labelsize=ticklabsize)
    ax.set_ylabel('Fixed Effects Coefficients', fontsize=18)
    
    # Autoformats the ticklabels for the xaxis
    fig.autofmt_xdate()
    
    sns.despine()

In [10]:
def Coefpresplot2(regtable,regtable2, intercept=False, title='Regression Coefficients', size='big', ylimits=(), ymultiple=0.5, ticklabsize=25, n_ET_predictors = 0):

    # Import itertools so that we can iterate through the colours
    import itertools
    
    # Import locators so that we can tidy up the yaxis
    from matplotlib.ticker import MultipleLocator, FormatStrFormatter
    
    # rounding function to get edges to even 0.5 values
    def round_to(n, precision):
        correction = 0.5 if n >= 0 else -0.5
        return int( n/precision+correction ) * precision

    def round_to_5(n):
        return round_to(n, 0.5)

    # Set seaborn style for the plot
    sns.set(style='white')
    
    # Generate the figure
    if size=='big':
        fig = plt.figure(figsize=[20,8])
    elif size=='long':
        fig = plt.figure(figsize=[20,4])
    elif size=='narrow':
        fig = plt.figure(figsize=[10,4])
        
    fig.suptitle(title, fontsize=20)
    gs = GridSpec(1,1,bottom=0.18,left=0.18,right=0.82)
    ax = fig.add_subplot(gs[0,0])
    

    # Set axis limits based on whether to include the intercept or not    
    
       
    
    if intercept == True:
        
        XLim = (0.75, len(regtable.columns) + 0.25)
        
        #Determine the Ymax and Ymin for both regresions results
        if round_to_5(regtable.loc['CImin', :].min()) < round_to_5(regtable2.loc['CImin', :].min()):
            Ymin = round_to_5(regtable.loc['CImin', :].min()-0.5)
        else :
            Ymin = round_to_5(regtable2.loc['CImin',:].min()-0.5)
        if round_to_5(regtable.loc['CImax', :].max()) > round_to_5(regtable2.loc['CImax', :].max()):
            Ymax = round_to_5(regtable.loc['CImax', :].max()+0.5)
        else :
            Ymax = round_to_5(regtable2.loc['CImax', :].max()+0.5)      

        YLim = (Ymin, Ymax)    
    else:
        XLim = (0.75, len(regtable.columns) - 0.75)
        
        #Determine the Ymax and Ymin for both regresions results
        if round_to_5(regtable.loc['CImin', regtable.columns[1]:].min()) < round_to_5(regtable2.loc['CImin', regtable.columns[1]:].min()):
            Ymin = round_to_5(regtable.loc['CImin', regtable.columns[1]:].min()-0.5)
        else :
            Ymin = round_to_5(regtable2.loc['CImin',regtable.columns[1]:].min()-0.5)
        if round_to_5(regtable.loc['CImax', regtable.columns[1]:].max()) > round_to_5(regtable2.loc['CImax', regtable.columns[1]:].max()):
            Ymax = round_to_5(regtable.loc['CImax', regtable.columns[1]:].max()+0.5)
        else :
            Ymax = round_to_5(regtable2.loc['CImax', regtable.columns[1]:].max()+0.5)      
        
        YLim = (Ymin, Ymax)
    
    if ylimits != ():
        YLim = ylimits
    ax.set_xlim(XLim)
    ax.set_ylim(YLim)
    
    # Draw a line through the 0-value on the y-axis
    line = ax.plot(XLim, [0, 0], color='black', ls='--', alpha = 0.5, lw=3)
    
    
    # If intercept is true, plot the coefficient for the intercept
    # both regtable should have the same regressors (and in the same order)

    if intercept == True:
        Coefficients = regtable.columns
 #       Coefficients2 = regtable2.columns
    else:
        Coefficients = regtable.columns[1:]
#        Coefficients2 = regtable2.columns[1:]
        
    # Determine the colours for the coefficients based on the n_ET_variable
    n_predictors = len(Coefficients)
    n_non_ET_predictors = n_predictors - n_ET_predictors
    
    colourlist = ['#000000'] * n_non_ET_predictors + ['#03719c'] * n_ET_predictors
    
        
    # Plot all the coefficients with 95% CI
    position = 0
    for Coefficient in Coefficients:
        position += 1
        ax.plot(position-0.1, regtable.loc['coefficient', Coefficient], marker='o', ms=8, color='blue',label = 'Like')
        ax.plot(position+0.1, regtable2.loc['coefficient', Coefficient], marker='X', ms=8, color='red', label = 'Dislike')


        ax.errorbar(position-0.1, regtable.loc['coefficient', Coefficient],
                    yerr=regtable.loc['se', Coefficient]*1.96, lw=2, color='blue')
        ax.errorbar(position+0.1, regtable2.loc['coefficient', Coefficient],
                    yerr=regtable2.loc['se', Coefficient]*1.96, lw=2, color='red')
        
        if position == 1:
            ax.legend( prop={'size': 20})

 
    # Setting the x-axis major tick's location
    ax.set_xticks(range(1, position+1))
    
    # set the y-axis major tick position
    ax.yaxis.set_major_locator(MultipleLocator(ymultiple))

    # Setting the x-axis major tick's label
    if intercept == True:
        ax.set_xticklabels(regtable.columns, rotation=40)
    else: 
        ax.set_xticklabels(regtable.columns[1:], rotation=40)
    
    ax.tick_params(axis='both', which='major', labelsize=ticklabsize)
    ax.set_ylabel('Fixed Effects Coefficients', fontsize=18)
    
    # Autoformats the ticklabels for the xaxis
    fig.autofmt_xdate()

    sns.despine()

In [11]:
def regtable(fix, fix_se, names):
    fixed_betas = DF(np.array(fix)); fixed_betas = fixed_betas.transpose(); fixed_betas.columns = names
    fixed_betas.loc[1] = np.array(fix_se)
    fixed_betas.loc[2] = fixed_betas.loc[0] - (fixed_betas.loc[1]*1.96)
    fixed_betas.loc[3] = fixed_betas.loc[0] + (fixed_betas.loc[1]*1.96)
    fixed_betas.index = ['coefficient', 'se', 'CImin', 'CImax']
    return fixed_betas